# Importações

In [1]:
from calendar import month

import pandas as pd
import datetime
import calendar
import locale
import numpy as np
import glob
import os
import math

from jinja2 import Undefined

locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')


'pt_BR.UTF-8'

# Carregar datasets

In [10]:
consumo = pd.read_csv('./dados/consumo.csv', sep=";", header=0)
consumo["DATA INICIO"] = pd.to_datetime(consumo['DATA INICIO'], format='%d/%m/%Y')
consumo["DATA FIM"] = pd.to_datetime(consumo['DATA FIM'], format='%d/%m/%Y')
feriados = pd.read_csv('dados/calendarios.csv', sep=";", header=0)
cursos_sup = pd.read_csv('./dados/cursos_superiores.csv', sep=";", header=0)
cursos_tec = pd.read_csv('./dados/cursos_tecnicos.csv', sep=";", header=0)


# Agrupamento dos Datasets


In [11]:

# Agrupando os dados por campus 
grupos_consumo = consumo.groupby('CAMPUS')
grupos_feriados = feriados.groupby('CAMPUS')
grupos_cursos_sup = cursos_sup.query('Modalidade == "Educação Presencial"').groupby('CAMPUS')
grupos_cursos_tec = cursos_tec.groupby('CAMPUS')

# Criando subdatasets para cada campus 
subdf_consumo = {campus: dados for campus, dados in grupos_consumo}
subdf_feriados = {campus: dados for campus, dados in grupos_feriados}
subdf_cursos_sup = {campus: dados for campus, dados in grupos_cursos_sup}
subdf_cursos_tec = {campus: dados for campus, dados in grupos_cursos_tec}



# Normalização dos meses de Consumo para o mesmo periodo

In [29]:
dataframes = []
dataframes_ignorados = []
for campus, df_consumo in subdf_consumo.items():
    dt_min = subdf_consumo[campus]["DATA INICIO"].min()
    dt_max = subdf_consumo[campus]["DATA FIM"].max()

    datas = pd.date_range(start=dt_min, end=dt_max)
    df = pd.DataFrame({'DATA': datas})
    df["CAMPUS"] = campus

    for index, row in df_consumo.iterrows():
        dt_inicio = row['DATA INICIO']
        dt_fim = row['DATA FIM']
        nr_dias = (dt_fim - dt_inicio).days + 1

        mascara = (df['DATA'] >= pd.to_datetime(dt_inicio)) & (df['DATA'] <= pd.to_datetime(dt_fim))
        df.loc[mascara, "CONSUMO"] = row["CONSUMO"] / nr_dias

    if "PARANAGUÁ" in campus:
        dataframes_ignorados.append(df)
    else:
        dataframes.append(df)

df_merged = pd.merge(dataframes_ignorados[0], dataframes_ignorados[1], on='DATA', how='inner', suffixes=('_df1', '_df2'))
df_merged["CONSUMO"] = df_merged["CONSUMO_df1"].fillna(0) + df_merged["CONSUMO_df2"].fillna(0)
df_merged["CAMPUS"] = "PARANAGUÁ"
df_merged = df_merged.drop(["CONSUMO_df1", "CONSUMO_df2", "CAMPUS_df1", "CAMPUS_df2"],axis=1)
dataframes.append(df_merged)

df_consumo_normalizado = pd.concat(dataframes, ignore_index=True)

df_consumo_normalizado.to_csv("./dados/dados_consumo_normalizados.csv", sep=";", decimal=".", index=False)
grupos_consumo_normalizado = df_consumo_normalizado.groupby('CAMPUS')
subdf_consumo_normalizado = {campus: dados for campus, dados in grupos_consumo_normalizado}


# Criação dos Datasets conforme Calendários Acadêmicos

In [44]:

for campus, df_feriados in subdf_consumo_normalizado.items():
    dt_min = pd.to_datetime(subdf_consumo_normalizado[campus]["DATA"]).min()
    dt_max = pd.to_datetime(subdf_consumo_normalizado[campus]["DATA"]).max()

    dt_inicio = datetime.date(year=2010, month=1, day=1)
    dt_fim = datetime.date.today()

    datas = pd.date_range(start=dt_inicio, end=dt_fim)
    df = pd.DataFrame({'DATA': datas})
    df["CAMPUS"] = campus

    for index, row in df.iterrows():
        data = row['DATA'].date()
        dia = data.day
        mes = data.month
        ano = data.year

        rows_data_fixa = feriados.loc[feriados["DATA INICIO"] == f"{dia}/{mes}"]
        rows_data_flexivel = feriados.loc[feriados["DATA INICIO"] == f"{dia:02}/{mes:02}/{ano:04}"]

        if not rows_data_fixa.empty:
            for _, row_data_fixa in rows_data_fixa.iterrows():
                if campus in row_data_fixa["CAMPUS"] or row_data_fixa["CAMPUS"] == "*":
                    [dia_final_fixo, mes_final_fixo] = str(row_data_fixa["DATA FIM"]).split("/")
                    data_final_fixa = datetime.date(ano, int(mes_final_fixo), int(dia_final_fixo))
                    mascara = (df['DATA'] >= pd.to_datetime(data)) & (df['DATA'] <= pd.to_datetime(data_final_fixa))
                    df.loc[mascara, row_data_fixa["MOTIVO"]] = 1

        if not rows_data_flexivel.empty:
            for _, row_data_flexivel in rows_data_flexivel.iterrows():
                if campus in row_data_flexivel["CAMPUS"] or row_data_flexivel["CAMPUS"] == "*":
                    [dia_final_flexivel, mes_final_flexivel, ano_final_flexivel] = str(
                        row_data_flexivel["DATA FIM"]).split("/")
                    data_final_flexivel = datetime.date(int(ano_final_flexivel), int(mes_final_flexivel),
                                                        int(dia_final_flexivel))
                    mascara = (df['DATA'] >= pd.to_datetime(data)) & (df['DATA'] <= pd.to_datetime(data_final_flexivel))
                    df.loc[mascara, row_data_flexivel["MOTIVO"]] = 1

    df = df.fillna(0)
    df = df.loc[(df['DATA'] >= dt_min) & (df['DATA'] <= dt_max)]
    df.to_csv(f"./dados/calendarios/{campus}.csv", index=False, sep=";", decimal=".")


# Criação do Dataset de Dados Temporais

In [ ]:
pasta = './dados/calendarios/'

# Obter uma lista de todos os arquivos .csv na pasta
arquivos_csv = glob.glob(os.path.join(pasta, "*.csv"))
# Criar uma lista para armazenar os DataFrames
dataframes = []

# Iterar sobre a lista de arquivos .csv e ler cada um em um DataFrame
for arquivo in arquivos_csv:
    df = pd.read_csv(arquivo, sep=";", header=0)
    df['CAMPUS'] = os.path.basename(arquivo.replace(".csv", ""))
    df['DATA'] = pd.to_datetime(df['DATA'])
    df['DIA_DA_SEMANA'] = df['DATA'].dt.strftime('%a')
    df['MÊS'] = df['DATA'].dt.strftime('%b')
    df['ANO'] = df['DATA'].dt.strftime('%Y')
    df = pd.get_dummies(df, columns=['DIA_DA_SEMANA', "MÊS", "ANO"])
    df = df.fillna(False)
    df[df.drop(["CAMPUS", "DATA"], axis=1).columns] = df[df.drop(["CAMPUS", "DATA"], axis=1).columns].astype(int)
    dataframes.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_completo = pd.concat(dataframes, ignore_index=True)
df_completo = df_completo.fillna(0)

df_completo.to_csv("./dados/dados_temporais.csv", sep=";", decimal=".", index=False)


# Criação dos Datasets conforme Cursos Superiores (E-MEC)

In [51]:
for campus, df_cursos_sup in subdf_cursos_sup.items():
    dt_min = pd.to_datetime(subdf_consumo_normalizado[campus]["DATA"]).min()
    dt_max = pd.to_datetime(subdf_consumo_normalizado[campus]["DATA"]).max()

    datas = pd.date_range(start=pd.to_datetime(df_cursos_sup["Início Funcionamento"], format='%d/%m/%Y').min(),
                          end=datetime.date.today())
    df = pd.DataFrame({'DATA': datas})
    df["CAMPUS"] = campus
    df["TURMAS_GRAD_MATUTINO"] = 0
    df["TURMAS_GRAD_VESPERTINO"] = 0
    df["TURMAS_GRAD_NOTURNO"] = 0
    df["TURMAS_POS"] = 0

    for index, row in df_cursos_sup.iterrows():
        dt_inicio = pd.to_datetime(row["Início Funcionamento"], format='%d/%m/%Y')
        if row["Grau"] in ["Bacharelado", "Tecnológico", "Licenciatura"]:
            if row["Quantitativo de Vagas - Integral"] > 0:
                n_anos = row["Integralização - Turno Noturno (ANOS)"]
                df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)), "TURMAS_GRAD_MATUTINO"] += math.ceil(n_anos)
                df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)), "TURMAS_GRAD_VESPERTINO"] += math.ceil(n_anos)

                if campus != "LONDRINA - NORTE":
                    for i in range(1, math.floor(n_anos)):
                        df.loc[(df['DATA'] <= pd.to_datetime(dt_inicio) + pd.DateOffset(
                            years=i)), "TURMAS_GRAD_MATUTINO"] -= 1
                        df.loc[(df['DATA'] <= pd.to_datetime(dt_inicio) + pd.DateOffset(
                            years=i)), "TURMAS_GRAD_VESPERTINO"] -= 1

                    if n_anos % 1 == 5:
                        df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio) + pd.DateOffset(years=n_anos) and df[
                            'DATA'] <= pd.to_datetime(dt_inicio) + pd.DateOffset(
                            years=math.ceil(n_anos))), "TURMAS_GRAD_MATUTINO"] -= 1
                        df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio) + pd.DateOffset(years=n_anos) and df[
                            'DATA'] <= pd.to_datetime(dt_inicio) + pd.DateOffset(
                            years=math.ceil(n_anos))), "TURMAS_GRAD_VESPERTINO"] -= 1

            else:
                if row["Quantitativo de Vagas - Matutino"] > 0:
                    n_anos = row["Integralização - Turno Matutino (ANOS)"]
                    df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)), "TURMAS_GRAD_MATUTINO"] += math.ceil(n_anos)
                    if campus != "LONDRINA - NORTE":
                        for i in range(1, math.floor(n_anos)):
                            df.loc[(df['DATA'] <= pd.to_datetime(dt_inicio) + pd.DateOffset(
                                years=i)), "TURMAS_GRAD_MATUTINO"] -= 1

                if row["Quantitativo de Vagas - Vespertino"] > 0:
                    n_anos = row["Integralização - Turno Vespertino (ANOS)"]
                    df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)), "TURMAS_GRAD_VESPERTINO"] += math.ceil(n_anos)
                    if campus != "LONDRINA - NORTE":
                        for i in range(1, math.floor(n_anos)):
                            df.loc[(df['DATA'] <= pd.to_datetime(dt_inicio) + pd.DateOffset(
                                years=i)), "TURMAS_GRAD_VESPERTINO"] -= 1

            if row["Quantitativo de Vagas - Noturno"] > 0:
                n_anos = row["Integralização - Turno Noturno (ANOS)"]
                df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)), "TURMAS_GRAD_NOTURNO"] += math.ceil(n_anos)
                if campus != "LONDRINA - NORTE":
                    for i in range(1, math.floor(n_anos)):
                        df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) & (df['DATA'] <= pd.to_datetime(dt_inicio) + pd.DateOffset(
                            years=i)), "TURMAS_GRAD_NOTURNO"] -= 1

        elif row["Grau"] == "Especialização":
            df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)), "TURMAS_POS"] += 1
        elif row["Grau"] == "Mestrado":
            # Acrescenta 2 turmas pois o mestrado geralmente tem duração bianual e novas admissões anuais
            df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)), "TURMAS_POS"] += 2
            # Remove 1 turma pois no primeiro ano geralmente só há 1 turma (mestrados são 2 anos)
            df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) & (df['DATA'] <= pd.to_datetime(dt_inicio) + pd.DateOffset(years=1)), "TURMAS_POS"] -= 1

    df = df.fillna(0)
    df = df.loc[(df['DATA'] >= dt_min) & (df['DATA'] <= dt_max)]
    df.to_csv(f"./dados/cursos/{campus}.csv", index=False, sep=";", decimal=".")

